In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" #model will be trained on GPU 0

In [2]:
'''Import libraries and files'''
import glob
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
#os.chdir('C://Users//Ron//Desktop//ML_proj/Brain_cancer/data')
'''Insert the .mat files data directory'''
#os.chdir('') #Please put the directory in which data is present
files=glob.glob('*.mat')

In [3]:
'''Initial data extraction from matlab dictionary'''
import h5py
import cv2 
X=[]
Y=[]
data=[]
count=0
#a = 1
for file in files:
    with h5py.File(file,'r') as f:
        variables=f.items()
        for var in variables:
            name= var[0]
            data=var[1]
            if (np.array(data['tumorMask']).shape[1] == 256):
                count+=1
                #print 
                (count)
            else:
                d=data['image']
                #Resizing for performance and computational time
                X.append(cv2.resize(np.array(d),dsize=(64,64),interpolation=cv2.INTER_AREA).flatten())

                #X.append(np.array(d))
                #X.append(np.array(data['tumorMask']))
                Y.append(data['label'][0][0])
                
                #a = data['label'][0][0]
                


In [4]:
#np.array(X).shape , np.array(Y).shape

((3049, 4096), (3049,))

In [5]:
# import cv2
# width = 64
# height = 3049
# dim = (width, height)
 
# # resize image
# resized = cv2.resize(np.array(X), dim, interpolation = cv2.INTER_AREA)

In [6]:
# resized = (np.array(resized) / 16).astype(np.float32)

In [7]:
# np.array(X).flatten().shape

In [8]:
X = np.array(X)

In [9]:
from keras.utils import to_categorical
Y=np.array(Y)-1
Y = to_categorical(Y)

Using TensorFlow backend.


In [10]:
#Splitting the data
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(X,Y, test_size=0.25, random_state=0)

In [11]:
'''Creating a multilayer perceptron layered network'''
import keras
#Sequential model
model = keras.models.Sequential()
model.add(keras.layers.normalization.BatchNormalization(input_shape=tuple([x_train.shape[1]])))
model.add(keras.layers.core.Dense(32, activation='relu'))
model.add(keras.layers.core.Dropout(rate=0.5))
model.add(keras.layers.normalization.BatchNormalization())
model.add(keras.layers.core.Dense(32, activation='relu'))
model.add(keras.layers.core.Dropout(rate=0.5))
model.add(keras.layers.normalization.BatchNormalization())
model.add(keras.layers.core.Dense(32, activation='relu'))
model.add(keras.layers.core.Dropout(rate=0.5))
model.add(keras.layers.core.Dense(3,   activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer='rmsprop',metrics=["accuracy"])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 4096)              16384     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                131104    
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)  

In [12]:
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=0, mode='auto')

In [13]:
'''Training'''
model.fit(x_train, y_train, batch_size=64, epochs=200, verbose=2, callbacks=[callback_early_stopping])

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
 - 2s - loss: 1.2485 - acc: 0.4125
Epoch 2/200
 - 0s - loss: 0.9660 - acc: 0.5464
Epoch 3/200
 - 0s - loss: 0.8233 - acc: 0.6181
Epoch 4/200
 - 0s - loss: 0.7337 - acc: 0.6465
Epoch 5/200
 - 0s - loss: 0.7019 - acc: 0.6802
Epoch 6/200
 - 0s - loss: 0.6342 - acc: 0.7113
Epoch 7/200
 - 0s - loss: 0.5823 - acc: 0.7454
Epoch 8/200
 - 0s - loss: 0.5302 - acc: 0.7537
Epoch 9/200
 - 0s - loss: 0.5248 - acc: 0.7638
Epoch 10/200
 - 0s - loss: 0.4838 - acc: 0.7988
Epoch 11/200
 - 0s - loss: 0.4984 - acc: 0.7817
Epoch 12/200
 - 0s - loss: 0.4535 - acc: 0.8128
Epoch 13/200
 - 1s - loss: 0.4435 - acc: 0.8185
Epoch 14/200
 - 0s - loss: 0.4128 - acc: 0.8171
Epoch 15/200
 - 1s - loss: 0.4121 - acc: 0.8325
Epoch 16/200
 - 0s - loss: 0.3822 - acc: 0.8548
Epoch 17/200
 - 1s - loss: 0.4093 - acc: 0.8351
Epoch 18/200
 - 1s - loss: 0.3686 - acc: 0.8657
Epoch 19/200
 - 0s - loss: 0.3712 - acc: 0.8570
Epoch 20/200
 - 0s - loss: 0.3669 - acc: 0.8618
E

In [14]:
Final = model.evaluate(x_valid,y_valid)

763/763 [==============================] - 1s 742us/step


In [15]:
print('Test loss:', Final[0])
print('Test accuracy:', Final[1])

Test loss: 0.24908629397655377
Test accuracy: 0.9239842733893138


In [18]:
'''Evaluation'''
predicted_classes = model.predict(x_valid)

predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

Y_test = np.argmax(np.round(y_valid),axis=1)

from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(3)]
print(classification_report(Y_test, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.79      0.92      0.85       179
     Class 1       0.96      0.88      0.92       341
     Class 2       0.99      0.99      0.99       243

   micro avg       0.92      0.92      0.92       763
   macro avg       0.91      0.93      0.92       763
weighted avg       0.93      0.92      0.93       763

